# Structured streaming in Pyspark 
Everybody talks streaming nowadays – social networks, online transactional systems they all generate data. Data collection means nothing without proper and on-time analysis. In this new data age, we are privileged with the right tools to make the best use of our data. We can use structured streaming to take advantage of this and act quickly upon new trends, this could bring to insights unseen before.
Spark offers two ways of streaming:
• Spark Streaming
• Structured streaming (officially introduced with Spark 2.0, production-ready with Spark 2.2)
Let’s add a few words for both streaming options below.
# Spark Streaming
Spark Streaming is a separate library in Spark which provides a basic abstraction layer called Discretized Stream or DStream, it processes continuously flowing streaming data by breaking it up into discrete chunks. DStream is the original, RDD (Resilient Distributed Dataset) based streaming API for Spark.
Spark streaming has the following problems:
• Difficult – not simple to build streaming pipelines which support late data or fault tolerance. All of them are achievable but they need some extra development work.
• Inconsistent Integration- API used to generate batch processing (RDD, Dataset) is different than the API of streaming processing (DStream).
• Processing order – later generated data is processed before earlier generated data.
# Structured Streaming
Structured streaming is based on Dataframe and Dataset APIs, it is easier to implement and SQL queries are easily applied. Most importantly, Structured streaming incorporates the following features:
• Strong guarantees about consistency with batch jobs – the engine uploads the data as a sequential stream.
• Transactional integration with storage systems – transactional updates are part of the core API now, once data is processed it is only being updated, this provides a consistent snapshot of the data.
• Tight integration with the rest of Spark – Structured Streaming supports serving interactive queries on streaming state with Spark SQL and JDBC, and integrates with MLlib.
• Late data support – explicit support of “event time” to aggregate out of order data (late data) and bigger support for windowing and sessions, this avoids the problems Spark Streaming has with Processing Order.
# Watermarking

Firstly we will introduce a Watermark in order to handle late arriving data. Spark’s engine automatically tracks the current event time and can filter out incoming messages if they are older than time T. In our use case we want to filter out messages that just arrived but are more than 1 day old. We can do that with the following code
# output modes:

    Append mode - Only the new rows added to the Result Table since the last trigger will be outputted to the sink
    Complete Mode - The whole Result Table will be outputted to the sink after every trigger (only supported for aggregate queries)
    Update Mode - Only the rows in the Result Table that were updated since the last trigger will be outputted to the sink
    
https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#handling-late-data-and-watermarking
# Example
# Source Data
For this tutorial I’ve used open-source data for taxis in NYC:
https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page
The purpose of this blog is to introduce you to structured streaming using a simple scenario with CSV input data with specific dates and times which we will be able to use. The static set of files are used to emulate streaming taxi orders. I computed real-time metrics like peak time of taxi pickups and drop-offs, most popular boroughs for taxi demand.
For this exercise, I took one FHV Taxi CSV file – for January 2018 and split it into multiple smaller sized files.
The data is 11 CSV files – 10 with transactional records and one location mapping, each transactional CSV file has about 5000 rows. The collection of the files serve as an echo of what real data might be like.
I’ve done some testing in terms of storage – if you decide to use ADLS the structured streaming won’t be working since it requires uploading of new files in the streaming folder or editing while the streaming is on. 
For the below test I chose DBFS as a storage location.
All source files and Databricks notebook can be found on the following link:
https://github.com/VickyAugust10/PysparkStructuredStreaming
Shall we get started?
Let’s get a preview of our main source folder:


In [1]:
import os
import sys
import re
import time
import findspark
findspark.init()
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
from pyspark.context import SparkContext 

spark = SparkSession.builder.getOrCreate()#create spark session 
sc = spark.sparkContext#create sparkContext
from pyspark.sql.types import *
from pyspark.sql import Row
# from pyspark.sql.functions import *
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pyspark.sql.functions as func
import matplotlib.patches as mpatches
from operator import add
from operator import add
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint
import itertools
from pyspark.streaming import StreamingContext


In [2]:
from pyspark.sql.types import *

# Path to our 20 JSON files

inputPath = "C:/Users/rzouga/Downloads/Work Swiss/location-exploration/Recruitment-Challenge/streaming/"
# Explicitly set schema
schema = StructType([ StructField("time", TimestampType(), True),
                      StructField("customer", StringType(), True),
                      StructField("action", StringType(), True),
                      StructField("device", StringType(), True)])
from pyspark.sql.types import TimestampType, StringType, StructType, StructField



# Create DataFrame representing data in the JSON files
inputDF = (
  spark
    .read
    .schema(schema)
    .json(inputPath)
)

# Remove empty rows
inputDF = inputDF.dropna()

# Aggregate number of actions
actionsDF = (
  inputDF
    .groupBy(
       inputDF.action
    )
    .count()
)
actionsDF.cache()

# Create temp table named 'iot_action_counts'
actionsDF.createOrReplaceTempView("iot_action_counts")

In [3]:
# Create streaming equivalent of `inputDF` using .readStream
streamingDF = (
  spark
    .readStream
    .schema(schema)
    .option("maxFilesPerTrigger", 1)
    .json(inputPath)
)
# Stream `streamingDF` while aggregating by action
streamingActionCountsDF = (
  streamingDF
    .groupBy(
      streamingDF.action
    )
    .count()
)
# Is `streamingActionCountsDF` actually streaming?
streamingActionCountsDF.isStreaming

True

In [4]:
#spark.streams.active 

In [5]:
# this is used for testing 
spark.conf.set("spark.sql.shuffle.partitions", "2")
# View stream in real-time
query = (
  streamingActionCountsDF.writeStream.queryName("streamingActionCountsDF")\
.format("memory").outputMode("complete")\
.start())




In [ ]:
Now we have a streaming dataframe, but it is not writing anywhere. We need to stream to a certain destination using writestream() on our dataframe with concrete options:

In [6]:
from time import sleep
for x in range(5):
    spark.sql("select * from streamingActionCountsDF").show(3)
    sleep(2)

+------+-----+
|action|count|
+------+-----+
+------+-----+

+-----------+-----+
|     action|count|
+-----------+-----+
|       null|    1|
|low battery|  339|
|  power off|  347|
+-----------+-----+
only showing top 3 rows

+-----------+-----+
|     action|count|
+-----------+-----+
|       null|    2|
|low battery|  668|
|  power off|  678|
+-----------+-----+
only showing top 3 rows

+-----------+-----+
|     action|count|
+-----------+-----+
|       null|    3|
|low battery|  986|
|  power off| 1019|
+-----------+-----+
only showing top 3 rows

+-----------+-----+
|     action|count|
+-----------+-----+
|       null|    4|
|low battery| 1312|
|  power off| 1357|
+-----------+-----+
only showing top 3 rows



In [7]:
# this used for testing also 

"""# View stream in real-time
query = (
  streamingActionCountsDF
    .writeStream
    .format('console')\# parquet for real program 
    .outputMode("complete")\
    .start()
)

query.awaitTermination()"""


"""
query = (
  OrderByBoroughPerDayAndServiceType
    .writeStream
    .format("memory")        # memory = store in-memory table (for testing only in Spark 2.0)
    .queryName("counts")     # counts = name of the in-memory table            
    .outputMode("complete")  # complete = all the counts should be in the table
    .start()
)"""

'# View stream in real-time\nquery = (\n  streamingActionCountsDF\n    .writeStream\n    .format(\'console\')    .outputMode("complete")    .start()\n)\n\nquery.awaitTermination()'

In [9]:
query= locations.writeStream\
  .format('console')\
  .outputMode("append")\
  .start()




In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()

# Create DataFrame representing the stream of input lines from connection to localhost:9999
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

# Split the lines into words
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

# Generate running word count
wordCounts = words.groupBy("word").count()
wordCounts

DataFrame[word: string, count: bigint]

In [19]:
wordCounts.isStreaming

True

In [23]:
 # Start running the query that prints the running counts to the console
query1 =(wordCounts.writeStream.queryName("wordCounts")\
.format("memory").outputMode("complete")\
.start())

from time import sleep
for x in range(2):
    spark.sql("select * from wordCounts").show(2)
    sleep(2)

+----+-----+
|word|count|
+----+-----+
+----+-----+

+----+-----+
|word|count|
+----+-----+
+----+-----+



In [ ]:
query.awaitTermination()

In [ ]:
import os
print(os.environ['SPARK_HOME'])

In [ ]:
#https://fr.slideshare.net/databricks/deep-dive-into-stateful-stream-processing-in-structured-streaming-with-tathagata-das?from_action=save
# http://www.dcs.bbk.ac.uk/~dell/teaching/cc/book/databricks/spark-intro.pdf
#https://docs.databricks.com/_static/notebooks/structured-streaming-etl-kafka.html
#https://databricks.com/blog/2017/04/26/processing-data-in-apache-kafka-with-structured-streaming-in-apache-spark-2-2.html
# https://sparkbyexamples.com/category/spark/apache-spark-streaming/
# https://hackersandslackers.com/structured-streaming-in-pyspark/
#https://www.toptal.com/apache/apache-spark-streaming-twitter
#https://stackoverflow.com/questions/45411285/spark-structured-streaming-and-filters
# https://adatis.co.uk/structured-streaming-in-pyspark-using-databricks-2/
# https://github.com/h2oai/sparkling-water/tree/master/examples#step-by-step-through-weather-data-example
# https://blog.invivoo.com/structured-streaming-in-spark/
# https://github.com/falaybeg/SparkStreaming-Network-Anomaly-Detection/blob/master/k-means_network-anomaly.ipynb
#https://gist.github.com/rmoff/eadf82da8a0cd506c6c4a19ebd18037e
# https://databricks.com/blog/2017/04/04/real-time-end-to-end-integration-with-apache-kafka-in-apache-sparks-structured-streaming.html